The input state of the Lunar Lander consists of following components:

  1. Horizontal Position
  2. Vertical Position
  3. Horizontal Velocity
  4. Vertical Velocity
  5. Angle
  6. Angular Velocity
  7. Left Leg Contact
  8. Right Leg Contact

The actions of the agents are:
  1. Do Nothing
  2. Fire Main Engine
  3. Fire Left Engine
  4. Fire Right Engine


<img src="https://raw.githubusercontent.com/NeuromatchAcademy/course-content-dl/main/projects/static/lunar_lander.png">

In [ ]:
!pip install Box2D
!pip install box2d-py
!pip install gym[all]
!pip install gym[Box_2D]

In [ ]:
import gym
from gym.wrappers import Monitor
env = env=gym.make("LunarLander-v2")
state = env.reset() # reset environment to a new, random state
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))
action = env.action_space.sample() # Explore action space
print(action)

In [ ]:
import numpy as np
q_table = {}

In [ ]:
%%time
"""Training the agent"""
import random
from IPython.display import clear_output
# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1
# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()
    epochs, penalties, reward, = 0, 0, 0
    done = False
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
          try:
            action = np.argmax(q_table[state]) # Exploit learned values
          except:
            q_table[state]=[0,0,0,0]
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) #explain more
        old_value = q_table[state][action]
        try:
          next_max = np.max(q_table[next_state])
        except:
          q_table[state]=[0,0,0,0]
          next_max = np.max(q_table[next_state])
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        if state in q_table.keys():
          q_table[state][action] = new_value
        else:
          q_table[state]=[0,0,0,0]
          q_table[state][action] = new_value

        if reward == -10:
            penalties += 1
        state = next_state
        epochs += 1
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

In [ ]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100
frames1 = [] # for animation
for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    done = False
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        if reward == -10:
            penalties += 1
        # Put each rendered frame into dict for animation
        frames1.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
            }
        )

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")